In [1]:
from os import path,listdir
from datetime import datetime,timedelta
import time
import re
import pandas as pd
import numpy as np
import random
import pytz
from tqdm import tqdm
import bz2
import json
import glob
local_tz = 'Asia/Almaty'

In [2]:
IN_PATH='raw/bf/BASIC'
OUT_PATH='data/bf'
ID_UNDER=47972
ID_OVER=47973
ID_DRAW=58805

In [3]:
def read_match(eventId, path):
    di={}
    for f in listdir('{}/{}/'.format(path,eventId)):
        process_odds=False
        if not f.startswith('1.'):
            continue
        found=False
        with bz2.open('{}/{}/{}'.format(path, eventId, f), "rt") as bz_file:
            suspendedTime=0
            inplay=False
            for line in bz_file:
                if not found and not 'MATCH_ODDS' in line and not 'OVER_UNDER_25' in line and not 'HALF_TIME' in line:
                    break
                #print(eventId,f)
                found=True
                j=json.loads(line)
                odi={
                    'eventId':eventId,
                    'clk':datetime.fromtimestamp(j['pt']//1000)
                }
                for x in j['mc']:
                    if 'marketDefinition' in x.keys():
                        node=x['marketDefinition']
                        if node['inPlay'] and not inplay:
                            inplay=node['inPlay']
                            inplayTime=datetime.fromtimestamp(j['pt']//1000)
                        if node['marketType']=='MATCH_ODDS':
                            process_odds=True
                            di['eventId']=eventId
                            di['clk']=datetime.fromtimestamp(j['pt']//1000)
                            di['marketTime']= datetime.strptime(node['marketTime'][:-5],'%Y-%m-%dT%H:%M:%S')
                            di['countryCode']= node['countryCode'] if 'countryCode' in node.keys() else ''
                            di['openDate']= datetime.strptime(node['openDate'][:-5],'%Y-%m-%dT%H:%M:%S')
                            di['eventName']= node['eventName']
                            di['home_id']= node['runners'][0]['id']
                            di['home_name']= node['runners'][0]['name']
                            di['home_sp']= node['runners'][0]['sortPriority']
                            di['away_id']= node['runners'][1]['id']
                            di['away_name']= node['runners'][1]['name']
                            di['away_sp']= node['runners'][1]['sortPriority']
                            di['draw_id']= node['runners'][2]['id']
                            di['draw_name']= node['runners'][2]['name']
                            di['draw_sp']= node['runners'][2]['sortPriority']
                            if inplay:
                                di['inplayTime']=inplayTime
                        if node['marketType']=='OVER_UNDER_25':
                            process_odds=True
                            di['under_id']= node['runners'][0]['id']
                            di['under_name']= node['runners'][0]['name']
                            di['over_id']= node['runners'][1]['id']
                            di['over_name']= node['runners'][1]['name']
                        if node['marketType']=='HALF_TIME':
                            process_odds=False
                            if node['status']=="SUSPENDED":
                                suspendedTime=datetime.fromtimestamp(j['pt']//1000)
                            elif node['status']=="CLOSED":
                                di['halfTime']=suspendedTime
                    if 'rc' in x.keys() and process_odds:
                        for rc in x['rc']:
                            odi['ltp']= rc['ltp']
                            odi['id']= rc['id']
                            odi['inplay']=inplay
                            odds_changes.append(odi)
    match_changes.append(di)


In [15]:
d= datetime(2020, 11, 1)
end_date= datetime(2020, 12, 1)
while d<end_date:
    print(d)
    in_path='{}/{dt:%Y/%b/}{dt.day}/'.format(IN_PATH, dt=d)
    if path.isdir(in_path):# and d.day<10:
        match_changes=[]
        odds_changes=[]
        for f in tqdm(listdir(in_path)):
            read_match(int(f),in_path)
            #break
        out_path='{}/{:%Y-%b-%d}'.format(OUT_PATH, d)
        pd.DataFrame(match_changes).to_csv(out_path+'_matches.csv', index=False)
        pd.DataFrame(odds_changes).drop_duplicates().to_csv(out_path+'_odds.csv', index=False)
    d+=timedelta(days=1)

100%|██████████| 33/33 [00:03<00:00,  9.66it/s]


In [15]:
dfs=[]
for f in glob.glob(path.join('data/bf/', '2015*_matches.csv')):
    dfs.append(pd.read_csv(f))
    print(f)
df=pd.concat(dfs, axis=0)

data/bf\2015-Aug-01_matches.csv
data/bf\2015-Aug-02_matches.csv
data/bf\2015-Aug-03_matches.csv
data/bf\2015-Aug-04_matches.csv
data/bf\2015-Aug-05_matches.csv
data/bf\2015-Aug-06_matches.csv
data/bf\2015-Aug-07_matches.csv
data/bf\2015-Aug-08_matches.csv
data/bf\2015-Aug-09_matches.csv
data/bf\2015-Aug-10_matches.csv
data/bf\2015-Aug-11_matches.csv
data/bf\2015-Aug-12_matches.csv
data/bf\2015-Aug-13_matches.csv
data/bf\2015-Aug-14_matches.csv
data/bf\2015-Aug-15_matches.csv
data/bf\2015-Aug-16_matches.csv
data/bf\2015-Aug-17_matches.csv
data/bf\2015-Aug-18_matches.csv
data/bf\2015-Aug-19_matches.csv
data/bf\2015-Aug-20_matches.csv
data/bf\2015-Aug-21_matches.csv
data/bf\2015-Aug-22_matches.csv
data/bf\2015-Aug-23_matches.csv
data/bf\2015-Aug-24_matches.csv
data/bf\2015-Aug-25_matches.csv
data/bf\2015-Aug-26_matches.csv
data/bf\2015-Aug-27_matches.csv
data/bf\2015-Aug-28_matches.csv
data/bf\2015-Aug-29_matches.csv
data/bf\2015-Aug-30_matches.csv
data/bf\2015-Aug-31_matches.csv
data/bf\

In [138]:
df=pd.read_csv('data/matches_2015.csv', index_col=None)



In [143]:
df

,eventId,countryCode,openDate,eventName,home_id,home_name,away_id,away_name,inplayTime,halfTime
0,27501910.0,GB,2015-08-01 14:00:00,Burnley v Bradford,56299.0,Burnley,48444.0,Bradford,2015-08-01 20:01:38,2015-08-01 20:47:42
1,27505573.0,DE,2015-08-08 12:00:00,Rot-Weiss Ahlen v Schalke II,2486895.0,Rot-Weiss Ahlen,5340425.0,Schalke II,2015-08-08 18:00:28,2015-08-08 18:45:30
2,27506755.0,FR,2015-08-11 18:00:00,Le Havre v CA Bastia,70446.0,Le Havre,6601801.0,CA Bastia,2015-08-12 00:01:00,2015-08-12 00:46:01
3,27507664.0,TR,2015-08-14 19:00:00,Fenerbahce v Eskisehirspor,46726.0,Fenerbahce,2080746.0,Eskisehirspor,2015-08-15 18:43:14,2015-08-15 01:50:19
4,27507713.0,EE,2015-08-15 13:30:00,Tammeka Tartu v Levadia Tallinn,5407303.0,Tammeka Tartu,206564.0,Levadia Tallinn,2015-08-15 19:30:37,2015-08-15 20:17:29
...,...,...,...,...,...,...,...,...,...,...
49139,27553062.0,ES,2015-09-30 18:30:00,CF Torre Levante v UD Alzira,9355536.0,CF Torre Levante,5688358.0,UD Alzira,2015-10-01 00:39:10,2015-10-01 01:26:09
49140,27553063.0,ES,2015-09-30 18:00:00,Ontinyent CF v Elche Ilicitano,9320419.0,Ontinyent CF,10243224.0,Elche Ilicitano,2015-10-01 00:53:55,2015-10-01 01:39:58
49141,27553064.0,ES,2015-09-30 18:45:00,Espinardo Atletico v Lorca Deportivo CF,10215484.0,Espinardo Atletico,10243205.0,Lorca Deportivo CF,2015-10-01 00:46:21,2015-10-01 01:32:51
49142,27553066.0,ES,2015-09-30 18:00:00,UD Mutilvera v CD Cantolagua,9397043.0,UD Mutilvera,10356679.0,CD Cantolagua,2015-10-01 00:02:14,2015-10-01 00:47:27


In [117]:
df1.loc[df1['home_name']=='The Draw']

,eventId,countryCode,openDate,eventName,home_id,home_name,away_id,away_name,draw_id,draw_name,inplayTime,halfTime,ht,at


In [118]:
df=pd.concat([df1,df2], axis=0)

In [56]:
df=pd.read_csv('data/matches_2015.csv', index_col=None)

## Matches

In [3]:
dfs=[]
for y in range(2015,2021):
    for f in glob.glob(path.join('data/bf/days/', f'{y}*_matches.csv')):
        print(f)
        dfs.append(pd.read_csv(f))
        
df=pd.concat(dfs, axis=0)
df=df.loc[~df['home_id'].isna()]
df=df[['eventId','countryCode','openDate','eventName','home_id','home_name','away_id','away_name','draw_id','draw_name','inplayTime','halfTime']]
df=df[df['eventName'].str.contains(' v ')]
df[['ht','at']]=df.apply(lambda x: x.eventName.split(' v '), axis=1, result_type="expand")
df=df.reset_index(drop=True)
df1=df.loc[(df['draw_name']!='The Draw')]
df2=df.loc[(df['draw_name']=='The Draw')]
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_id']=df1['draw_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'away_id']=df1['home_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'home_id']=df1['draw_id']
df1.loc[(df1['away_name']=='The Draw') & (df1['at'].str.lower()==df1['home_name'].str.lower()),'home_name']=df1['ht']
df1.loc[(df1['away_name']=='The Draw') & (df1['ht'].str.lower()==df1['home_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_id']=df1['draw_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_name']=df1['ht']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'home_id']=df1['away_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'away_id']=df1['draw_id']
df1.loc[(df1['home_name']=='The Draw') & (df1['ht'].str.lower()==df1['away_name'].str.lower()),'away_name']=df1['at']
df1.loc[(df1['home_name']=='The Draw') & (df1['at'].str.lower()==df1['away_name'].str.lower()),'home_name']=df1['ht']
df=pd.concat([df1,df2], axis=0)
df.dropna(subset=['inplayTime'], inplace=True)
df=df[~((df['home_name']=='test1') | (df['away_name']=='test1') | (df['home_name']=='test2') | (df['away_name']=='test2'))]
df=df[['eventId','countryCode','openDate','eventName','home_id','home_name','away_id','away_name','inplayTime','halfTime']]
df.to_csv('data/bf/bf_matches.csv', index=False)

-Aug-10_matches.csv
data/bf/days\2019-Aug-11_matches.csv
data/bf/days\2019-Aug-12_matches.csv
data/bf/days\2019-Aug-13_matches.csv
data/bf/days\2019-Aug-14_matches.csv
data/bf/days\2019-Aug-15_matches.csv
data/bf/days\2019-Aug-16_matches.csv
data/bf/days\2019-Aug-17_matches.csv
data/bf/days\2019-Aug-18_matches.csv
data/bf/days\2019-Aug-19_matches.csv
data/bf/days\2019-Aug-20_matches.csv
data/bf/days\2019-Aug-21_matches.csv
data/bf/days\2019-Aug-22_matches.csv
data/bf/days\2019-Aug-23_matches.csv
data/bf/days\2019-Aug-24_matches.csv
data/bf/days\2019-Aug-25_matches.csv
data/bf/days\2019-Aug-26_matches.csv
data/bf/days\2019-Aug-27_matches.csv
data/bf/days\2019-Aug-28_matches.csv
data/bf/days\2019-Aug-29_matches.csv
data/bf/days\2019-Aug-30_matches.csv
data/bf/days\2019-Aug-31_matches.csv
data/bf/days\2019-Dec-01_matches.csv
data/bf/days\2019-Dec-02_matches.csv
data/bf/days\2019-Dec-03_matches.csv
data/bf/days\2019-Dec-04_matches.csv
data/bf/days\2019-Dec-05_matches.csv
data/bf/days\2019-

## Odds

In [204]:
cols=['eventId','clk','ltp','id','ip']
cols_noid=['eventId','clk','ltp','ip']
df_matches=pd.read_csv('data/bf/bf_matches.csv', index_col=None)
for y in range(2015,2021):
    dfs=[]
    for f in glob.glob(path.join('data/bf/days/', f'{y}*_odds.csv')):
        print(f)
        dfs.append(pd.read_csv(f))
        
    df=pd.concat(dfs, axis=0)
    df=df[df['eventId'].isin(df_matches['eventId'])]
    df['ip']=0
    df.loc[df['inplay'],'ip']=1
    df=df[cols]
    df_un=df.loc[df['id']==ID_UNDER][cols_noid]
    df_ov=df.loc[df['id']==ID_OVER][cols_noid]
    df_draw=df.loc[df['id']==ID_DRAW][cols_noid]
    df_home=pd.merge(df,df_matches, left_on=['eventId','id'], right_on=['eventId','home_id'])[cols_noid]
    df_away=pd.merge(df,df_matches, left_on=['eventId','id'], right_on=['eventId','away_id'])[cols_noid]
    #df_12=df.loc[(df['id']!=ID_UNDER) & (df['id']!=ID_OVER) & (df['id']!=ID_DRAW)]
    df_un.to_csv(f'data/bf_un_{y}.csv', index=False)
    df_ov.to_csv(f'data/bf_ov_{y}.csv', index=False)
    df_draw.to_csv(f'data/bf_draw_{y}.csv', index=False)
    df_home.to_csv(f'data/bf_home_{y}.csv', index=False)
    df_away.to_csv(f'data/bf_away_{y}.csv', index=False)
    #break

dds.csv
data/bf/days\2018-Sep-23_odds.csv
data/bf/days\2018-Sep-24_odds.csv
data/bf/days\2018-Sep-25_odds.csv
data/bf/days\2018-Sep-26_odds.csv
data/bf/days\2018-Sep-27_odds.csv
data/bf/days\2018-Sep-28_odds.csv
data/bf/days\2018-Sep-29_odds.csv
data/bf/days\2018-Sep-30_odds.csv
data/bf/days\2019-Apr-01_odds.csv
data/bf/days\2019-Apr-02_odds.csv
data/bf/days\2019-Apr-03_odds.csv
data/bf/days\2019-Apr-04_odds.csv
data/bf/days\2019-Apr-05_odds.csv
data/bf/days\2019-Apr-06_odds.csv
data/bf/days\2019-Apr-07_odds.csv
data/bf/days\2019-Apr-08_odds.csv
data/bf/days\2019-Apr-09_odds.csv
data/bf/days\2019-Apr-10_odds.csv
data/bf/days\2019-Apr-11_odds.csv
data/bf/days\2019-Apr-12_odds.csv
data/bf/days\2019-Apr-13_odds.csv
data/bf/days\2019-Apr-14_odds.csv
data/bf/days\2019-Apr-15_odds.csv
data/bf/days\2019-Apr-16_odds.csv
data/bf/days\2019-Apr-17_odds.csv
data/bf/days\2019-Apr-18_odds.csv
data/bf/days\2019-Apr-19_odds.csv
data/bf/days\2019-Apr-20_odds.csv
data/bf/days\2019-Apr-21_odds.csv
data/b

In [181]:
df_matches=pd.read_csv('data/bf_matches.csv', index_col=None)

